<a href="https://colab.research.google.com/github/Thayyebsalim/music_analyisis/blob/master/Music_Sentimental_analysis_NLP_by_Tone_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The idea of this code is to find the mood of the song in any way possible helping us to segregate songs better.

1.   First part is gonna be scraping the lyrics from any website or API 
2.   Finding the vibe of the song with sentimental analysis
3.   Fetch and mp3 version and check the mood with transfer learning hopefully or something similar



In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvidia.com/com

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("brown")
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

We need to scrape the song lyrics from websites or API to use here

I am using selenium here and the speed is not the best at the moment so I might switch to scrapy or use an API

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


def get_track(): # Gets track name from user and converts it so that we can use it in a link
  track_name = input("Enter a track name: ")
  track_name_link = track_name.rstrip().replace(" ","%20").lower()
  return track_name_link


def scrape_tracklist(track_link): # Scrapes the page for list of tracks with similar names or such 
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get("https://www.musixmatch.com/search/" + track_link + "/tracks")
  element = driver.find_element_by_class_name("box-content")
  link_list = [i.get_attribute('href') for i in element.find_elements_by_tag_name('a')]
  driver.close()
  lyric_links=[i for i in link_list if re.search("/lyrics/",i)] #could do it in one line but for the sake of clarity, I'm gonna do two
  lyric_links=[i for i in lyric_links if not re.search("/add",i)]
  return lyric_links


def scrape_lyrics(lyric_links_list,choice): # Scrapes the lyrics element
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get(lyric_links_list[choice])
  lyric_elem=driver.find_elements_by_class_name('mxm-lyrics__content')
  lyric_elem= [i.find_element_by_tag_name('span') for i in lyric_elem] 
  lyric_elem= [i.get_attribute('innerHTML') for i in lyric_elem]
  driver.close()
  return lyric_elem


def select_track(artist_track_list): # Lets the user select a track
  sel=0
  for track in artist_track_list:
    print(f"{track[0]+1} {track[1]} - {track[2]}")
  sel=int(input('Enter the number of your desired track: '))-1
  return sel


def get_lyric_element(tracklist,choice=0): # Cleans up the lyric element and returns a cleaned list(array) with just lyrics inside
  lyric_links_list = scrape_tracklist(track_link)
  artist_track_list = [[lyric_links_list.index(x), x.split("/")[4],x.split('/')[5]] for x in lyric_links_list]
  print(artist_track_list)
  if len(artist_track_list) > 1 and choice != "default":
    choice = select_track(artist_track_list)
  elif len(artist_track_list) == 0 :
    return "Sorry! no such song"
  else:
    choice=0
  lyric_element = scrape_lyrics(lyric_links_list,choice)
  return " ".join(lyric_element)



In [ ]:
track_name= "when i was your man"
track_link=track_name.rstrip().replace(" ","%20").lower()
#track_link=get_track() #uncomment to activate function

In [ ]:
lyric_element = get_lyric_element(track_link,"default")
lyric_element

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


[[0, 'Bruno-Mars', 'When-I-Was-Your-Man'], [1, 'Andrew-Wyatt-Philip-Lawrence-Bruno-Mars-Ari-Levine-Martin-Ermen', 'When-I-Was-Your-Man'], [2, 'Bruno-Mars-feat-Daniel-Philo', 'When-I-Was-Your-Man-Acoustic-Cover'], [3, 'Boyce-Avenue-feat-Fifth-Harmony', 'When-I-Was-Your-Man'], [4, 'Rod-Stewart', 'When-I-Was-Your-Man'], [5, 'The-Temptations', 'When-I-Was-Your-Man'], [6, 'Jasmine-Thompson', 'When-I-Was-Your-Man'], [7, 'Thomas-Rhett', 'When-I-Was-Your-Man'], [8, 'PelleK', 'When-I-Was-Your-Man'], [9, 'BTOB', '%EB%82%B4%EA%B0%80-%EB%8B%88-%EB%82%A8%EC%9E%90%EC%98%80%EC%9D%84-%EB%95%8C'], [10, 'MAX-36', 'When-I-Was-Your-Man-Acoustic'], [11, 'Anthem-Lights', 'Treasure-When-I-Was-Your-Man'], [12, 'BTOB', 'When-I-Was-Your-Man'], [13, 'Lu-Robertinho', 'When-I-Was-Your-Man-Quando-Amanhecer'], [14, 'Fer-Palacio', 'When-I-Was-Your-Man'], [15, 'Rockabye-Baby', 'When-I-Was-Your-Man'], [16, 'Tyler-Ward', 'When-I-Was-Your-Man-acoustic'], [17, 'Various-Artists', 'When-I-Was-Your-Man'], [18, 'Twinkle-Twink

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


"Same bed but it feels just a little bit bigger now\nOur song on the radio but it don't sound the same\nWhen our friends talk about you All it does is just tear me down\n'Cause my heart breaks a little\nWhen I hear your name\n\nIt all just sounds like ooh-ooh, ooh-ooh\nMmm, too young, too dumb to realize\n\nThat I should've bought you flowers\nAnd held your hand\nShould've gave you all my hours\nWhen I had the chance\nTake you to every party\n'Cause all you wanted to do was dance\nNow my baby's dancing\nBut she's dancing with another man\n\nMy pride, my ego, my needs, and my selfish ways\nCaused a good strong woman like you\nTo walk out my life\nNow I never, never get to clean up the mess I made\nAnd it haunts me every time I close my eyes\n\nIt all just sounds like ooh-ooh, ooh-ooh\nMmm, too young, too dumb to realize\n\nThat I should've bought you flowers\nAnd held your hand\nShould've gave you all my hours\nWhen I had the chance\nTake you to every party\n'Cause all you wanted to do 

In [ ]:
"""
tracks_queue=[]
for track in tracks_queue:
  
"""

'\ntracks_queue=[]\nfor track in tracks_queue:\n  \n'

The first part is done, web scraper should work fine. I did not implement error cases for now becuase I want to focus on the next two parts of the project


In [ ]:
#joiner
def convert_arr_to_str(arr):
  return " ".join(arr)

In [8]:
lyrics_as_array= filter_raw_lyrics(lyric_element)
lyrics=convert_arr_to_str(lyrics_as_array) #returns a single string as lyrics
lyrics,lyrics_as_array

NameError: ignored

In [ ]:
def filter_raw_lyrics(raw_lyr): #The scraped element has blanks and \n's lets filter those 
  return [i for i in raw_lyr.split("\n") if i != '']

In [ ]:
def swfilter(word): #function we use in filter()
  return word not in stopwords.words()

def remove_stopword(lyr_arr):
  return [" ".join(filter(swfilter,sentence.split())) for sentence in lyrics_as_array]

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def contraction(word): #function we use in map()
  word=word.lower()
  if word in contractions:
    return contractions[word]
  else: 
    return word

def remove_contraction(lyr_arr):
  return [" ".join(map(contraction, sentence.split())) for sentence in lyr_arr]

In [ ]:
#
# Returns all transformed arrays and further maps the arrays 

def get_lyric_arrays(lyr_elt):
  lyr_arr=filter_raw_lyrics(lyr_elt)
  return [lyr_arr,remove_stopword(lyr_arr),remove_contraction(lyr_arr)]

def get_lyrics(arr):
  return list(map(convert_arr_to_str,arr))

get_lyric_arrays(lyric_element) #returns a tuple with 3 items lyrics, lyrics without stop wordsm 
v=get_lyrics(get_lyric_arrays(lyric_element))
v
#"""

In [ ]:
lyrics_as_array_without_sw = remove_stopword(lyrics_as_array)
lyrics_without_sw = convert_arr_to_str(lyrics_as_array_without_sw)
lyrics_as_array_without_sw,lyrics_without_sw

In [ ]:
lyrics_as_array_without_contraction = remove_contraction(lyrics_as_array)
lyrics_without_contraction = convert_arr_to_str(lyrics_as_array_without_contraction)
lyrics_as_array_without_contraction,lyrics_without_contraction

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

def map_lemmatize(word):
  return wordnet_lemmatizer.lemmatize(word, pos="v")

x=" ".join(map(map_lemmatize,lyrics.split()))
x

In [ ]:
for i in v:
  print(Sentiment_score(i))
print(Sentiment_score(x))


Next we have to tackle the sentimantal analysis part. We aim to try bag of words then use NLTK on every line might be slow but I like to give prominence to accuracy

In [ ]:
lyric_tokens = word_tokenize(lyrics)
blob= TextBlob(lyrics_without_sw)
print ((blob.polarity*100))
blob

In [ ]:
listofpol=[]
sumofpol=0
for i in lyr:
  blobs=TextBlob(i)
  pol=blobs.polarity
  listofpol.append(pol)
  sumofpol=sumofpol+pol
  print(f'{i} - {pol}')
print ("Average:",(sumofpol)*100/len(lyr),"%")

In [ ]:
sum([i for i in listofpol if i!=0])/len([i for i in listofpol if i!=0])


In [ ]:
mn=["i am bad","i am not bad","i am good","i am not good"]
for i in lyr:
  print(TextBlob(i).polarity)
  print(TextBlob(i).ngrams(2))



In [ ]:
w=""
f=TextBlob(lyr[5]).ngrams(2)
for word in f:
  t=" ".join(word)
  w+=" "+t


In [ ]:
TextBlob(lyr[5]).polarity,TextBlob(w).polarity

Kinda done with this part. Looks okay for now. Now we got to see if we can get an mp3 version of the song and see if the song sounds sad or happy. 

In [ ]:
!pip install watson_developer_cloud

In [ ]:
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey="hzkkbjivKNHfoi3cFlMBOPG6-YHLc-7CAQKwXn8Rt7JT", # Use your API key here
    url="https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/512aa79c-a8ba-43fa-b9d5-a4382099b06f" # paste the url here
    )
def Sentiment_score(input_text): 
    # Input text can be sentence, paragraph or document
    response = natural_language_understanding.analyze (
    text = input_text,
    features = Features(sentiment=SentimentOptions())).get_result()
    # From the response extract score which is between -1 to 1
    res = response.get('sentiment').get('document').get('score')
    return res

In [ ]:
(Sentiment_score(lyrics))*100

In [ ]:
lyrics

In [ ]:
lyric_tokens_without_sw == lyrics_without_sw

In [ ]:
print(lyrics_without_sw)

In [ ]:
print(lyric_tokens_without_sw)

In [ ]:
if "you're" in contractions

In [ ]:
contractions["he's"]

In [ ]:
"he's" in contractions

In [ ]:
" ".join(filter(swfilter,lyrics.split())) == " ".join(lyrics_as_array_without_sw)